### Introduction to Neural Network Classification in Tensorflow

In [2]:
# DL needs
import tensorflow as tf
import tensorflow.keras as kr

# Data needs
import pandas as pd
from sklearn.model_selection import train_test_split

# Numerical computation needs
import numpy as np

# plotting needs
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# ensuring reproducibility
random_seed=42
tf.random.set_seed(random_seed)

**5. Improving Predictions on Fashion MNIST using normalized data**

In [3]:
from tensorflow.keras.datasets import fashion_mnist

(train_data, train_labels),(test_data,test_labels) = fashion_mnist.load_data()

In [4]:
# creating dictationary of labels:
labels = ['T-shirt/top', 
          'Trouser', 
          'Pullover',
          'Dress',   
          'Coat',    
          'Sandal',  
          'Shirt',   
          'Sneaker', 
          'Bag',     
          'Ankle boot']

In [5]:
# normalizing data
train_data_norm=(train_data-train_data.min())/train_data.max()
test_data_norm=(test_data-test_data.min())/test_data.max()
print(train_data_norm[0].min(),",",train_data_norm[0].max())
print(test_data_norm[0].min(),",",test_data_norm[0].max())

0.0 , 1.0
0.0 , 1.0


In [6]:
# one-hot encode labels
train_labels_ohe=tf.one_hot(train_labels,len(labels))
print(train_labels_ohe[:5])
test_labels_ohe=tf.one_hot(test_labels,len(labels))
print(test_labels_ohe[:5])

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(5, 10), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]], shape=(5, 10), dtype=float32)


In [7]:
# creating a model

class FashionNet(kr.Model):
    def __init__(self,lr=0.001,model_name="model_1",metrics=['accuracy']):
        super().__init__(name=model_name)
        # instantiating constructor of parent class

        # input layer:
        self.input_layer=tf.keras.Input(shape=[28,28],name='input_layer')
        
        # flattening the input
        self.flatten=kr.layers.Flatten(input_shape=(28,28),name='flatten')

        # hidden layer:
        self.fc1 = kr.layers.Dense(4,activation=None,name='hl1')
        self.fc2 = kr.layers.Dense(4,activation=None,name='hl2')
        # self.fc3 = kr.layers.Dense(4,activation=None,name='hl3')

        self.layer_activations={
            "flatten":None,
            "hl1":'ReLU',
            "hl2":'ReLU',
            "output_layer":"Softmax"
        }

        # output layer
        self.output_layer=kr.layers.Dense(10,activation=None,name='output_layer')


        # other hyper-params
        self.loss_function="categorical_crossentropy"
        self.optimizer=kr.optimizers.Adam(learning_rate=lr)
        self.eval_metrics=metrics

    def call(self,inputs):
        # forward propogation
        x=self.flatten(inputs)
        x=tf.nn.relu(self.fc1(x))
        x=tf.nn.relu(self.fc2(x))
        # x=tf.nn.relu(self.fc3(x))
        output=tf.nn.softmax(self.output_layer(x))
        return output

In [8]:
# fitting the model
model_1 = FashionNet(model_name='model_1',lr=0.001)
model_1(tf.keras.Input(shape=[28,28]))  
model_1.compile(loss=model_1.loss_function,optimizer=model_1.optimizer,metrics=model_1.eval_metrics)
model_1.summary()
history_2=model_1.fit(train_data_norm,train_labels_ohe,epochs=20,validation_data=(test_data_norm,test_labels_ohe))

/home/rudraksha14/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hl1 (Dense)                     │ (None, 4)              │         3,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hl2 (Dense)                     │ (None, 4)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │            50 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,210 (12.54 KB)

 Trainable params: 3,210 (12.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


2025-03-06 18:04:30.376184: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4963 - loss: 1.3995 - val_accuracy: 0.7278 - val_loss: 0.7893
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7417 - loss: 0.7440 - val_accuracy: 0.7656 - val_loss: 0.6848
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7749 - loss: 0.6477 - val_accuracy: 0.7815 - val_loss: 0.6297
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7910 - loss: 0.5996 - val_accuracy: 0.7914 - val_loss: 0.6071
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7985 - loss: 0.5775 - val_accuracy: 0.7981 - val_loss: 0.5945
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8050 - loss: 0.5632 - val_accuracy: 0.7990 - val_loss: 0.5871
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8099 - loss: 0.5531 - val_accuracy: 0.8012 - val_loss: 0.5818
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8126 - loss: 0.5454 - val_accurac

**9. Understanding patterns learnt by our model**

* by default the weights in the keras.layers.Dense is initialized using `kernel_initializer = 'glorot_uniform'` and the bias vectors is initialized to zeros.
* The bias vector dictates how much the pattern within the corresponding weights matrix should influence the next layer.

In [9]:
# Find the layers of model:
model_1.layers

[<Flatten name=flatten, built=True>,
 <Dense name=hl1, built=True>,
 <Dense name=hl2, built=True>,
 <Dense name=output_layer, built=True>]

In [10]:
# extract a particular layer and inspecting weights and biases
hl1=model_1.layers[1]
weight_1,bias_1=hl1.get_weights()

# printing shapes of weights and biases
print(weight_1.shape,bias_1.shape)
print(weight_1,bias_1)

(784, 4) (4,)
[[ 0.09309386 -1.0883762   0.21780913  0.5602282 ]
 [-1.0112921  -0.19426206  0.5343483   1.0279362 ]
 [ 0.3673576  -1.3152485   0.5681249   0.83580977]
 ...
 [-0.33126888 -0.13971567  0.04316235 -0.33810866]
 [-0.28372085 -0.14338706 -0.14433335 -0.6756599 ]
 [-0.02092897 -0.5624809   0.13253969  0.85101223]] [0.19855566 0.63483083 0.34453633 0.54407555]


In [12]:
import graphviz
import tensorflow as tf

def plot_custom_model(model, input_shape, show_shapes=True, show_activations=True,          
                      show_trainable_status=True,graph_size="8,8", dpi=100, node_width="1.5", node_height="0.5",ranksep="0.5", nodesep="0.3", title="Model Architecture", save_path=None):
    """
    Plots a detailed visualization of a subclassed Keras model with structured sections
    and different colours for each row while maintaining a single rectangle per layer.

    Parameters:
    - model: The Keras model to visualize.
    - input_shape: The expected input shape (excluding batch size).
    - show_shapes: Whether to display layer shapes.
    - show_activations: Whether to display activation functions.
    - show_trainable_status: Whether to display trainable status.
    - graph_size: The overall size of the graph (e.g., "8,8").
    - dpi: Resolution of the graph (higher = sharper but larger).
    - node_width: Width of each node.
    - node_height: Height of each node.
    - ranksep: Vertical spacing between layers.
    - nodesep: Horizontal spacing between nodes.
    - title: Title displayed at the top of the graph.
    - save_path: If specified, saves the plot as a PNG file.
    """
    dot = graphviz.Digraph(format='png')
    
    # Adjust graph properties
    dot.attr(size=graph_size, dpi=str(dpi), nodesep=nodesep, ranksep=ranksep)
    
    # Add title at the top
    dot.attr(label=f"<<B>{title}</B>>", labelloc="t", fontsize="16", fontcolor="black",fontweight='bold')

    prev_layer = None
    x = tf.keras.layers.Input(shape=input_shape)

    for layer in model.layers:
        layer_name = layer.name
        layer_type = type(layer).__name__

        # Get activation function
        activation = getattr(layer, "activation", None)
        activation_name = activation.__name__ if activation else "None"

        # Compute input & output shapes
        try:
            output_shape = layer.compute_output_shape(x.shape) if show_shapes else "N/A"
        except Exception:
            output_shape = "Unknown"

        activation_name = activation_name if show_activations else "N/A"
        # input_shape_str = str(x.shape) if show_shapes else "N/A"
        trainable_status = "Yes" if layer.trainable else "No"

        # Ensure each row exists properly even if not all options are enabled
        act_row = f'<TR><TD COLSPAN="3" BGCOLOR="lightgreen">Activation: {activation_name}</TD></TR>' if show_activations else ""

        shape_row = ""
        if show_shapes:
            shape_row += f'<TD BGCOLOR="lightyellow"><B>Input</B>: {str(x.shape)}</TD>\n'
            shape_row += f'<TD BGCOLOR="lightpink"><B>Output</B>: {output_shape}</TD>'
        else:
            shape_row += '<TD COLSPAN="2"></TD>'  # Maintain table structure

        train_stat_row = f'<TD BGCOLOR="lightgrey"><B>Trainable</B>: {trainable_status}</TD>' if show_trainable_status else ""

        # Ensure at least one row is always present
        if not (show_shapes or show_trainable_status):
            shape_row = '<TD COLSPAN="3"></TD>'

        # Table format with controlled spacing
        label = f"""<
        <TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
            <TR><TD COLSPAN="3" BGCOLOR="lightblue"><B>{layer_name}</B> ({layer_type})</TD></TR>
            {act_row}
            <TR>
                {shape_row}
                {train_stat_row}
            </TR>
        </TABLE>
        >"""

        # Create the node with adjusted width/height
        dot.node(layer_name, label=label, shape="plaintext", width=node_width, height=node_height)

        # Connect layers sequentially
        if prev_layer:
            dot.edge(prev_layer.name, layer_name)

        prev_layer = layer
        x = layer(x)  # Pass dummy input through each layer

    if save_path:
        dot.render(save_path, format="png", cleanup=True)

    return dot

# Example usage with a title:
dot_graph = plot_custom_model(model_1, input_shape=(28, 28), 
                              show_shapes=True, 
                              show_activations=True, 
                              show_trainable_status=True,
                              graph_size="6,6", dpi=300,  
                              node_width="1.2", node_height="0.5",  
                              ranksep="0.4", nodesep="0.2",  
                              title="model_1",
                              save_path="model")

dot_graph.view()

'model.png'